In [3]:
import pymupdf
import io
from PIL import Image
import pytesseract
import numpy as np
import cv2
from matplotlib import pyplot as plt

input_dir = "./static/input"
output_dir = "./static/output"
work_dir = ""

def raw_txt_from_imgpdf(in_file):
    docs = []
    doc = pymupdf.open(in_file) # open document
    full_text = ""
    name_file = in_file.split("/")[-1].split(".")[0]
    
    for e, page in enumerate(doc): # iterate through the pages
        pix = page.get_pixmap(dpi=300) # render page to an image
        pix_bytes = pix.tobytes()
 
        img = Image.open(io.BytesIO(pix_bytes))
        type_img = "original" 
        output_file = f"{output_dir}/{name_file}_{e}_{type_img}.png"
        # print(output_file)
        img.save(output_file, "PNG")        
        img = cv2.imread(output_file)
        
        # Rescaling
        # Rescale the image, if needed.
        img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
        
        # Convert to gray
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
        # Apply dilation and erosion to remove some noise
        kernel = np.ones((1, 1), np.uint8)
        img = cv2.dilate(img, kernel, iterations=1)
        img = cv2.erode(img, kernel, iterations=1)
        # Apply blur to smooth out the edges
        img = cv2.GaussianBlur(img, (5, 5), 0)
        
        # Apply threshold to get image with only b&w (binarization)
        img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        
        # Save the filtered image in the output directory
        type_img = "processed" 
        output_file = f"{output_dir}/{name_file}_{e}_{type_img}.png"
        # print(output_file)
        cv2.imwrite(output_file, img)
 
        # custom_config = r'--oem 3 --psm 6'
        custom_config = ""
        text = pytesseract.image_to_string(img, lang="Latin", config=custom_config)
        docs.append(text)
 
    return docs

file = f"{input_dir}/118498.pdf"

docs = raw_txt_from_imgpdf(file)
print(f"Pages: {len(docs)}")
 
# raw_txt = "\n\n".join(doc for doc in docs)
# raw_txt


KeyboardInterrupt: 